# 문서 군집화 - Opinion Review dataset

데이터가 파일 단위로 들어가 있음 => 어떻게 처리하면 좋을까?

In [2]:
import pandas as pd
import glob, os

path = 'OpinosisDataset1.0/topics' # path는 폴더에 들어간 파일들을 설정 
os.path.join(path, '*.data')       # 폴더에 들어간 확장자명이 '.data'로 동일
                                   # Windows - \\, Mac - /

'OpinosisDataset1.0/topics\\*.data'

In [3]:
# 파일이름이 리스트로 나와서 반복문을 사용할 수 있다. 
# asterisk: v* => v로 시작하는 것들, *.data => .data로 끝나는 것들
all_files = glob.glob(os.path.join(path, '*.data'))
all_files[:5]

['OpinosisDataset1.0/topics\\accuracy_garmin_nuvi_255W_gps.txt.data',
 'OpinosisDataset1.0/topics\\bathroom_bestwestern_hotel_sfo.txt.data',
 'OpinosisDataset1.0/topics\\battery-life_amazon_kindle.txt.data',
 'OpinosisDataset1.0/topics\\battery-life_ipod_nano_8gb.txt.data',
 'OpinosisDataset1.0/topics\\battery-life_netbook_1005ha.txt.data']

In [4]:
file = all_files[0]
file

'OpinosisDataset1.0/topics\\accuracy_garmin_nuvi_255W_gps.txt.data'

In [5]:
file.split('\\')[-1].split('.')[0]

'accuracy_garmin_nuvi_255W_gps'

In [6]:
# 리스트 컴프리헨션으로 만들 수 있다!
filename_list = []
opinion_text = []
for file in glob.glob(os.path.join(path, '*.data')):        # all_files라는 변수가 헷갈릴 수 있음!
    # 파일을 읽기 위한 3단계: open(<filename>) -> read(), write() : 작업 -> close() : 작업종료
        # open() -> read(), write()만 하고 close()를 안한다면?
        #   사용자가 혼자일 경우 문제는 없지만, 대형 서버의 경우, 서버의 사용자가 access한다면, 시스템에 있는 리소스가 계속 열려 있는 상태로 문제를 일으킬 소지가 있다
        #   이를 방지하기 위해 python에서는 open()을 with로 쓰면 with문을 벗어나는 순간 파일이 자동으로 close된다 == 리소스 낭비를 막을 수 있다.
    with open(file, encoding='latin1') as f:   # with는 keyword                                   
        text = f.read()     # with문 안에서 인덴테이션? 해야 한다.
    opinion_text.append(text)
    filename = file.split('\\')[-1].split('.')[0]
    filename_list.append(filename)

df = pd.DataFrame({'filename':filename_list, 'opinion':opinion_text})
df.head(3)

,filename,opinion
0,accuracy_garmin_nuvi_255W_gps,", and is very, very accurate .\n but for the m..."
1,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and ve..."
2,battery-life_amazon_kindle,After I plugged it in to my USB hub on my com...
